##### EEEL 4220 Final Project
## Wind Farm Sizing and Siting on IEEE-14 System

4. 14-bus 5-scenario, with wind planning. Finally, assemble all scenarios together to produce the full planning model.

In [23]:
# Import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

In [24]:
# Load data
b_df = pd.read_csv('buses.csv')
g_df = pd.read_csv('generators.csv')
l_df = pd.read_csv('branches.csv')
d_df = pd.read_csv('demand_scenarios.csv')
cfs_df = pd.read_csv('wind_cf_scenarios.csv')
y_df = pd.read_csv('wind_candidates.csv')

In [25]:
# Parameters
c2 = g_df['c2'].to_numpy().reshape(-1,1) # quadratic cost coefficients for generator g
c1 = g_df['c1'].to_numpy().reshape(-1,1) # linear cost coefficients for generator g
c0 = g_df['c0'].to_numpy().reshape(-1,1) # fixed cost coefficients for generator g
Pmin = g_df['Pmin_MW'].to_numpy().reshape(-1,1) # minimum generation limits for generator g
Pmax = g_df['Pmax_MW'].to_numpy().reshape(-1,1) # maximum generation limits for generator g
x = l_df['x_pu'].to_numpy().reshape(-1,1) # reactance of transmission line l
fmax = l_df['rateA_MW'].to_numpy().reshape(-1,1) # thermal limit of line l
D = d_df.pivot(index='bus_id',columns='scenario_id',values='Pd_MW').to_numpy() # demand at bus b in scenario s
CF = cfs_df['capacity_factor'].to_numpy().reshape(1,-1) # wind capacity factor in scenario s
omega = cfs_df['probability'].to_numpy() # probability (weight) of scenario s
ybar = pd.merge(b_df, y_df, on='bus_id', how='left')['max_capacity_MW'].fillna(0).to_numpy().reshape(-1,1) # maximum installable wind capacity at bus b (0 if not candidate)
Cinv = 0.5 # investment cost per MW of installed wind capacity ($/MW)

# Decision variables
p = cp.Variable((len(g_df), len(cfs_df)), nonneg=True) # ps = power output of generator g under scenario s
f = cp.Variable((len(l_df), len(cfs_df))) # fs = power flow on line l under scenario s
theta = cp.Variable((len(b_df), len(cfs_df))) # thetas = voltage angle at bus b (radians)
w = cp.Variable((len(b_df), len(cfs_df)), nonneg=True) # ws = wind generation at bus b under scenario s
y = cp.Variable((len(b_df),1), nonneg=True) # installed wind capacity at bus b (MW)

In [26]:
# Initialize incidence matrices
# Generator–bus
# Ggb = 1 if generator g is connected to bus b, 0 otherwise.
G = np.zeros((len(g_df), len(b_df)))

# Line–bus
# Alb = 1 if bus b is the sending (“from”) end of line l, if bus b is the receiving (“to”) end of line l, 0 otherwise.
A = np.zeros((len(l_df), len(b_df)))

# Populate incidence matrices
# Generator–bus
for gen_index in range(len(g_df)):
    bus_id = g_df.loc[gen_index, 'bus_id']
    bus_index = bus_id - 1
    G[gen_index, bus_index] = 1

# Line–bus
for line_index in range(len(l_df)):
    from_bus_id = l_df.loc[line_index, 'from_bus']
    to_bus_id = l_df.loc[line_index, 'to_bus']
    from_bus_index = from_bus_id - 1
    to_bus_index = to_bus_id - 1
    A[line_index, from_bus_index] = 1
    A[line_index, to_bus_index] = -1

In [27]:
# Define constraints
# Initialize an empty constraint set
con = [] 

# power balance
con.append((A.T @ f) + D == (G.T @ p) + w)

# generator output limits
con.append(p <= Pmax)  # maximum generation

# set reference angle
con.append(theta[0] == 0)

# DC power flow
con.append(f == (A @ theta) / x)

# line flow limits
con.append(f <= fmax)  # maximum line flow
con.append(f >= -fmax)  # minimum line flow
# wind generation limits
con.append(w <= y @ CF)  # wind generation cannot exceed installed capacity

# wind capacity limits
con.append(y <= ybar)  # maximum installed capacity

# Define objective function
obj = cp.Minimize(cp.sum(cp.multiply(c2, cp.square(p)) + cp.multiply(c1, p) + c0, axis=0) @ omega + (Cinv * cp.sum(y)))

# Solve problem
prob = cp.Problem(obj, con)
prob.solve(solver = "HIGHS");

In [28]:
# Convert results to df
ICT = y.value.round(1)
GT = p.value.round(1)
WT = w.value.round(1)
LT = f.value.round(1)
MCT = (con[0].dual_value / omega).round(1)

ICT = pd.DataFrame(ICT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'],
                  columns=['Installed_Capacity_MW'])
MCT = pd.DataFrame(MCT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'],
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
GT = pd.DataFrame(GT, index = ['G1', 'G2', 'G3', 'G4', 'G5'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
WT = pd.DataFrame(WT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
LT = pd.DataFrame(LT, index = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])

In [29]:
# Print results
print("Total generation in the system for each scenario:")
print((GT.sum(axis=0) + WT.sum(axis=0)).round(1).to_list())
print("\n Total demand in the system for each scenario:") 
print(D.sum(axis=0).round(1))

print("\n Objective value (Total Cost): ")
print(obj.value)

print("\n Installed wind capacity results: ")
print(ICT.round(1))

print("\n Thermal generator dispatch results: ")
print(GT.round(1))

print("\n Wind dispatch results: ")
print(WT.round(1))

print("\n Line flow results: ")
print(LT.round(1))

print("\n Locational Marginal Prices: ")
print(MCT.round(1))

Total generation in the system for each scenario:
[233.1, 259.0, 284.9, 246.1, 271.9]

 Total demand in the system for each scenario:
[233.1 259.  284.9 246.  271.9]

 Objective value (Total Cost): 
455.0103955766357

 Installed wind capacity results: 
     Installed_Capacity_MW
B1                     0.0
B2                   150.0
B3                     0.0
B4                     0.0
B5                   120.0
B6                     0.0
B7                     0.0
B8                     0.0
B9                   179.8
B10                    0.0
B11                    0.0
B12                    0.0
B13                  120.0
B14                    0.0

 Thermal generator dispatch results: 
      S1    S2   S3    S4   S5
G1  38.0  16.6  0.0  53.0  0.0
G2  22.4  11.6  0.0  29.8  0.0
G3  18.8   5.9  0.0  27.8  0.0
G4  20.5  12.5  0.0  26.1  0.0
G5  19.4  13.0  0.0  23.9  0.0

 Wind dispatch results: 
       S1    S2    S3    S4    S5
B1    0.0   0.0   0.0   0.0   0.0
B2   30.0  52.5  75.0  

In [30]:
# Print results
print("Total generation in the system for each scenario:")
print((GT.sum(axis=0) + WT.sum(axis=0)).round(1).to_list())
print("\n Total demand in the system for each scenario:") 
print(D.sum(axis=0).round(1))

print("\n Installed wind capacity results: ")
print(ICT.round(1))

print("\n Thermal generator dispatch results: ")
print(GT.round(1))

print("\n Wind dispatch results: ")
print(WT.round(1))

print("\n Line flow results: ")
print(LT.round(1))

print("\n Locational Marginal Prices: ")
print(MCT.round(1))

Total generation in the system for each scenario:
[233.1, 259.0, 284.9, 246.1, 271.9]

 Total demand in the system for each scenario:
[233.1 259.  284.9 246.  271.9]

 Installed wind capacity results: 
     Installed_Capacity_MW
B1                     0.0
B2                   150.0
B3                     0.0
B4                     0.0
B5                   120.0
B6                     0.0
B7                     0.0
B8                     0.0
B9                   179.8
B10                    0.0
B11                    0.0
B12                    0.0
B13                  120.0
B14                    0.0

 Thermal generator dispatch results: 
      S1    S2   S3    S4   S5
G1  38.0  16.6  0.0  53.0  0.0
G2  22.4  11.6  0.0  29.8  0.0
G3  18.8   5.9  0.0  27.8  0.0
G4  20.5  12.5  0.0  26.1  0.0
G5  19.4  13.0  0.0  23.9  0.0

 Wind dispatch results: 
       S1    S2    S3    S4    S5
B1    0.0   0.0   0.0   0.0   0.0
B2   30.0  52.5  75.0  22.5  82.3
B3    0.0   0.0   0.0   0.0   0.0
B4    